In [69]:
import torch
import mss.tools
import math
import numpy as np
import cv2
import mouse
import keyboard
import time
import pydirectinput
from time import sleep
import ctypes

In [70]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\Admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-5-5 torch 1.11.0 CUDA:0 (NVIDIA GeForce RTX 3080, 10240MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [71]:



def getcoords(enemy):
    try:
        return [float(results.xyxy[0][enemy, 0]),
                float(results.xyxy[0][enemy, 1]),
                float(results.xyxy[0][enemy, 2]),
                float(results.xyxy[0][enemy, 3])]
    except IndexError:
        return None


def move_mousey(y):
    #mouse.move(0, y, absolute=False, duration=0)

    ctypes.windll.user32.mouse_event(
        ctypes.c_uint(0x0001),
        ctypes.c_uint(0),
        ctypes.c_uint(y),
        ctypes.c_uint(0),
        ctypes.c_uint(0)
    )


def move_mousex(x):
    #mouse.move(x, 0, absolute=False, duration=0)

    ctypes.windll.user32.mouse_event(
        ctypes.c_uint(0x0001),
        ctypes.c_uint(x),
        ctypes.c_uint(0),
        ctypes.c_uint(0),
        ctypes.c_uint(0)
    )

In [85]:
with mss.mss() as sct:
    # Use the first monitor, change to desired monitor number
    dimensions = sct.monitors[1]
    SQUARE_SIZE = 600

    # Part of the screen to capture
    monitor = {"top": int((dimensions['height'] / 2) - (SQUARE_SIZE / 2)),
               "left": int((dimensions['width'] / 2) - (SQUARE_SIZE / 2)),
               "width": SQUARE_SIZE,
               "height": SQUARE_SIZE}

    while True:

        start_time = time.time()  # start time of the loop

        # Screenshot
        sct_img = sct.grab(monitor)

        # Convert to
        img = np.array(sct_img)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Inference
        results = model(img)
        model.conf = 0.6

    # Reading and determining the closest enemy to crosshair

        # Get number of enemies / length of the .xyxy[0] array
        enemyNum = results.xyxy[0].shape[0]

        # Reset distances array to prevent repeating items
        distances = []

        # Cycle through enemies and draw lines from center to center of their head
        for i in range(enemyNum):
            enemy = getcoords(i)

            x1 = enemy[0]
            x2 = enemy[2]
            y1 = enemy[1]
            y2 = enemy[3]

            centerX = (x2 - x1) / 2 + x1
            centerY = (y2 - y1) / 2 + y1

            distance = math.sqrt(((centerX - 300) ** 2) + ((centerY - 300) ** 2))
            distances.append(distance)

            cv2.line(results.imgs[0], (int(centerX), int(centerY)), (int(SQUARE_SIZE/2), int(SQUARE_SIZE/2)), (255, 255, 255), 1, cv2.LINE_AA)

        # Get the shortest distance from the array (distances)
        closest = 1000



        # Get the smallest value of the array
        for i in range(0, len(distances)):
            if distances[i] < closest:
                closest = distances[i]
                closestEnemy = i

        # Getting the coordinates of the closest enemy
        if len(distances) != 0:
            enemyCoords = getcoords(closestEnemy)

            x1 = enemyCoords[0]
            x2 = enemyCoords[2]
            y1 = enemyCoords[1]
            y2 = enemyCoords[3]

            Xenemycoord = (x2 - x1) / 2 + x1
            Yenemycoord = (y2 - y1) / 2 + y1

    # Moving mouse to enemy

            difx = Xenemycoord - (SQUARE_SIZE / 2)
            dify = Yenemycoord - (SQUARE_SIZE / 2)

        '''if keyboard.is_pressed('/'):
            if difx < 0:
                move_mousex(int(difx * 2))

            elif difx > 0:
                move_mousex(int(abs(difx) * 2))

            if dify < 0:
                move_mousey(int(dify * 2))

            elif dify > 0:
                move_mousey(int(abs(dify) * 2))'''

        if mouse.is_pressed(button='left') and keyboard.is_pressed('/'):
            if difx < 0:
                move_mousex(int(difx * 2))

            elif difx > 0:
                move_mousex(int(abs(difx) * 2))

            if dify < 0:
                move_mousey(int(dify * 2))

            elif dify > 0:
                move_mousey(int(abs(dify) * 2))


        # Display the picture
        results.display(render=True)
        cv2.imshow('', results.imgs[0])

        # Press "q" to quit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            sct.close()
            break

        print("FPS: ", 1.0 / (time.time() - start_time))  # FPS = 1 / time to process loop

FPS:  21.719428521124936
FPS:  19.98182043390851
FPS:  32.228920939596286
FPS:  31.220395251032787
FPS:  20.814474787726603
FPS:  20.814371495211155
FPS:  20.814474787726603
FPS:  31.222022063749648
FPS:  31.221557242816733
FPS:  31.222022063749648
FPS:  31.22109243572375
FPS:  31.22132483754029
FPS:  31.22178965155316
FPS:  34.45098441850723
FPS:  28.54588517137179
FPS:  27.00253653511878
FPS:  23.234566807001993
FPS:  20.81406162381583
FPS:  20.814681375833096
FPS:  31.221557242816733
FPS:  31.22178965155316
FPS:  31.22109243572375
FPS:  31.220395251032787
FPS:  29.386487679448464
FPS:  33.30239944103028
FPS:  20.81426820372089
FPS:  31.222022063749648
FPS:  20.814578081267246
FPS:  20.81406162381583
FPS:  31.22178965155316
FPS:  31.222022063749648
FPS:  31.22109243572375
FPS:  38.426267956610964
FPS:  26.292290911826285
FPS:  20.81406162381583
FPS:  31.22178965155316
FPS:  31.221557242816733
FPS:  31.221557242816733
FPS:  20.814474787726603
FPS:  20.81426820372089
FPS:  31.220860037

In [83]:
if mouse.is_pressed(button='left'):
    print(1)